In [1]:
from glob import glob
import os
from sklearn.model_selection import train_test_split
from datasets import Dataset
from datasets import load_dataset

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
bert_name = "emilyalsentzer/Bio_ClinicalBERT"

In [3]:
import json, requests
from bs4 import BeautifulSoup
from shutil import copyfile

classes = ['hasAttr']

def read_sample(files):
    file_dict = {'text':[],'label':[]}
    for file in files:
        with open(file) as f:
            dataset = f.readlines()
        for sample in dataset:
            file_dict['text'].append(sample.split('\t')[4])
            file_dict['label'].append(int(sample.split('\t')[5].strip()))
        
    return file_dict

In [4]:
train_files = glob(f'/Users/yr255/Downloads/kiwi_development/data/RE/train/*.txt')
valid_files = glob(f'/Users/yr255/Downloads/kiwi_development/data/RE/dev/*.txt')

In [5]:
train_dict = read_sample(train_files)
valid_dict = read_sample(valid_files)

In [6]:
train = Dataset.from_dict(train_dict)
valid = Dataset.from_dict(valid_dict)

In [7]:
imdb = load_dataset("imdb",cache_dir = '/Users/yr255/Downloads/kiwi_development/')

all_dataset = imdb
all_dataset['train']=train
all_dataset['valid']=valid
all_dataset.pop('test')
all_dataset.pop('unsupervised')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 50000
})

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(bert_name)

In [11]:
def preprocess_function(examples):
    return tokenizer(examples["text"],truncation=True, max_length=512)
tokenized_dataset = all_dataset.map(preprocess_function, batched=True)
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
device = torch.device("mps")
model = AutoModelForSequenceClassification.from_pretrained(bert_name, num_labels=2).to(device)

from sklearn.metrics import f1_score
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return {'f1':f1_score(labels, predictions, average='weighted')}


Map:   0%|          | 0/1656 [00:00<?, ? examples/s]

Map:   0%|          | 0/1656 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
seqeval = evaluate.load("seqeval")

ImportError: cannot import name 'load_metric' from 'evaluate' (/opt/anaconda3/envs/m3/lib/python3.10/site-packages/evaluate/__init__.py)

In [12]:
training_args = TrainingArguments(
    output_dir=f"./models/RE/",
    learning_rate=1e-6,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    evaluation_strategy="epoch",
    num_train_epochs=20,
    weight_decay=0.01,
    load_best_model_at_end=True,
    save_strategy = "epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['valid'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics = compute_metrics 
)

#trainer.train()

/opt/anaconda3/envs/m3/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/gw/rzv539755mx1ncdf30k8lvbszl3zcm/T/ipykernel_35947/2655075283.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### TEST

In [13]:
import json, requests
from bs4 import BeautifulSoup
from shutil import copyfile
import pandas as pd
classes = ['hasAttr','negative']

def read_sample(file):
    file_dict = {'text':[],'label':[]}
    with open(file) as f:
        dataset = f.readlines()
    for sample in dataset:
        file_dict['text'].append(sample.split('\t')[4])
        file_dict['label'].append(int(sample.split('\t')[5].strip()))
        
    return file_dict

In [15]:
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
device = torch.device("mps")
model = AutoModelForSequenceClassification.from_pretrained('/Users/yr255/Downloads/kiwi_development/BERT/models/RE/', num_labels=2).to(device)

In [16]:
%%time
from torch.utils.data import DataLoader

# Define batch size
batch_size = 100

# Loop over the datasets
for set in ['i2b2']:
    print(set)
    test_file = f'../data/RE/test/RE_BERT_{set}_test.txt'
    test_dict = read_sample(test_file)
    test = Dataset.from_dict(test_dict)
    
    # Create DataLoader for batching
    test_loader = DataLoader(test, batch_size=batch_size)

    allpreds = []
    alllabels = []
    
    # Loop over batches of data
    for batch in test_loader:
        # Tokenize batch of texts
        inputs = tokenizer(batch['text'], truncation=True, padding=True, max_length=512, return_tensors="pt").to(device)
        labels = torch.tensor(batch['label']).unsqueeze(1).to(device)  # Adjust labels for batch

        # Forward pass through the model
        outputs = model(**inputs, labels=labels)
        loss, logits = outputs[:2]

        # Store predictions and true labels
        alllabels.extend(batch['label'])
        allpreds.extend(logits.argmax(dim=1).cpu().numpy())  # Get the predicted class indices


i2b2


<timed exec>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


RuntimeError: MPS backend out of memory (MPS allocated: 14.91 GB, other allocations: 5.27 GB, max allowed: 20.40 GB). Tried to allocate 250.78 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).